In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CourseWork/23F/CS4635/hw3

/content/drive/MyDrive/CourseWork/23F/CS4635/hw3


In [12]:
import json
import random
from collections import defaultdict
class CobwebNode():
  def __init__(self):
    self.instance_count = 0
    self.attribute_value = defaultdict(int)
    self.children=[]
    self.label = None # Label 이렇게 Initiate하는게 맞는지 확인 해보기

  def increment(self, attribute, value):
    self.attribute_value[(attribute, value)] += 1

  def decrement(self, attribute, value):
    self.attribute_value[(attribute, value)] -= 1

class LearningSystem(object):
	"""
	Set up a learning system class, i.e., the approach you will apply.
	Feel free to create any functions under the class.

	Two functions are REQUIRED:
	- LearningSystem.fit(train_set, train_labels)
	- LearningSystem.predict(instance)
	"""

	def __init__(self):
		"""
		You can have any other parameters here.
		"""
		self.root = CobwebNode()


	def fit(self, train_set, train_labels):
		"""
		Train the system with this function.
		Do NOT set any input variable other than train_set and train_labels for this function.
		========
		train_data: training data without labels. It should be a list of dictionaries.
		train_labels: A list of labels of training data. Each label has the same index as its data in train_data.
					  All labels are binary (0, 1).
		"""
		for instance in train_set:
      self.insert(self.root, instance)

  def insert(self, node, instance, label):
    node.instance_count +=1
    for attribute, value in instance.items():
      node.increment(attribute, value)

    best_cu = -1
    best_child = None

    for child in node.children:
      cu = concept_utility(node)
      if cu > best_cu:
        best_cu = cu
        best_child = child

    if best_child:
      node.insert(best_child, instance, label)
    else:
      new_child = CobwebNode()
      for attribute, value in instance.items():
        new_child.increment(attribute, value)
      new_child.instance_count = 1
      new_child.label = label
      node.children.append(new_child)

  def concept_utility(node):
    cu = 0
    for (attribute, value), count in node.attribute_value.items():
      prob = count / node.instance_count
      cu += prob * (prob - 1/node.instance_count)
    return cu


	def predict(self, instance):
		"""
		Predict the label of an instance (a single dictionary) given.
		"""
		return _predict_recursive(self.root, instance)

  def _predict_recursive(self, node, instance):
    if not node.children:
      return node.label

    best_cu = -1
    best_child = None

    for child in node.children:
      cu = concept_utility(child)
      if cu > best_cu:
        best_cu = cu
        best_child = child

    return _predict_recursive(best_child, instance)


# Then you need to make the separate define_model() function,
# so you can have any initialized parameter defined.
# Autograder will define the model object based on your define_model() function.
def define_model():
	return LearningSystem()


"""
The following codes are for testing with sample data only.
REMEMBER TO REMOVE THEM BEFORE SUBMISSION.
=============================================================================
In this homework, a sample collection of shuffled data is provided.
You might try with the data provided like in the following before submission:
"""

def test(model, test_set_data, test_labels):
	correct = 0
	for i in range(len(test_set_data)):
		pred = model.predict(test_set_data[i])
		if pred == test_labels[i]:
			correct += 1
	accuracy = correct / len(test_set_data)
	return accuracy

IndentationError: ignored

In [ ]:
sample_set_data = json.load(open('hw3-sample_set_data.json'))
sample_set_labels = json.load(open('hw3-sample_set_labels.json'))

train_set = sample_set_data[:-500]
train_labels = sample_set_labels[:-500]
test_set = sample_set_data[-500:]
test_labels = sample_set_labels[-500:]

model = define_model()
model.fit(train_set, train_labels)
print("Accuracy:", test(model, test_set, test_labels))